In [1]:
%matplotlib inline
import sys
import numpy as np
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Polygon
import matplotlib.colors as colors
import matplotlib.cm as cmx
import pandas as pd

In [8]:
# generate list of fields for SUMMER
# generate grid of 0.26x0.26 degree fields for SUMMER
# NOTE, not exactly 1x1, TODO: fix
#    we want a few arc-minute overlap between fields
#        ...how to calculate field corners?
plot1 = 0
plot2 = 0
rapoints = []
decpoints = []
ra1points, ra2points, ra3points, ra4points = [], [], [], []
dec1points, dec2points, dec3points, dec4points = [], [], [], []

# calculate field size in degrees
plate_scale = 0.034 # as/um
pix_size = 13.5 # um
pix_size_as = plate_scale * pix_size
num_pix = 2048
field_size = num_pix * pix_size_as  /3600 # degrees on sky
print('Field size', field_size)

half_size = field_size / 2
num_decs = 180 / field_size
print(num_decs, type(num_decs))
decs = np.linspace(90,-89,int(num_decs))
decsB = decs - half_size
decsT = decs + half_size
for i in range(len(decs)):
    circ = np.cos(decs[i]*np.pi/180.)
    radiffB = half_size/np.cos(decsB[i]*np.pi/180)
    radiffT = half_size/np.cos(decsT[i]*np.pi/180)
    numpoints = int(360/field_size*circ+1)
    ras = np.linspace(0,360,numpoints)[:-1]
    #print(ras)
    ra1s = ras - radiffB
    ra2s = ras + radiffB
    ra4s = ras - radiffT
    ra3s = ras + radiffT
    ra1points = np.append(ra1points,ra1s)
    ra2points = np.append(ra2points,ra2s)
    ra3points = np.append(ra3points,ra3s)
    ra4points = np.append(ra4points,ra4s)
    dec1points = np.append(dec1points,np.ones(numpoints-1)*decsB[i])
    dec2points = np.append(dec2points,np.ones(numpoints-1)*decsB[i])
    dec3points = np.append(dec3points,np.ones(numpoints-1)*decsT[i])
    dec4points = np.append(dec4points,np.ones(numpoints-1)*decsT[i])
    decpoints = np.append(decpoints,np.ones(numpoints-1)*decs[i])
    rapoints = np.append(rapoints,ras)

print('Grid contains %d fields' % (len(rapoints)))
ra = coord.Angle(rapoints*u.degree)
#print ra.radian
#wrap = 180.
wrap = 360.
ra = ra.wrap_at(wrap*u.degree)
#print ra.radian
dec = coord.Angle(decpoints*u.degree)

if plot1:
    fig1 = plt.figure(figsize=(8,6))
    ax1 = fig1.add_subplot(111, projection="mollweide")
    ax1.scatter(ra.radian, dec.radian, 0.01)

ra1points = coord.Angle(ra1points*u.degree)
ra1points = ra1points.wrap_at(wrap*u.degree)
dec1points = coord.Angle(dec1points*u.degree)

ra2points = coord.Angle(ra2points*u.degree)
ra2points = ra2points.wrap_at(wrap*u.degree)
dec2points = coord.Angle(dec2points*u.degree)

ra3points = coord.Angle(ra3points*u.degree)
ra3points = ra3points.wrap_at(wrap*u.degree)
dec3points = coord.Angle(dec3points*u.degree)

ra4points = coord.Angle(ra4points*u.degree)
ra4points = ra4points.wrap_at(wrap*u.degree)
dec4points = coord.Angle(dec4points*u.degree)


Field size 0.26112
689.3382352941176 <class 'float'>
Grid contains 606845 fields


In [10]:
#field_id = np.arange(len(ra))
field_id = np.linspace(1,len(ra),len(ra))
entry = np.linspace(0,len(ra)-1,len(ra))
ebv = np.ones(len(ra))
#print(*field_id)
data = {}
data['field_id'] = field_id
data['ra'] = ra.degree
data['dec'] = dec.degree
c = coord.SkyCoord(ra=ra,dec=dec,frame='icrs')
data['l'] = c.galactic.l.degree
data['b'] = c.galactic.b.degree
data['grid_id'] = np.zeros(len(ra), dtype=np.int)
data['eclLon'] = c.geocentricmeanecliptic.lon.degree #geocentricmeanecliptic???
data['eclLat'] = c.geocentricmeanecliptic.lat.degree
data['entry'] = entry
data['ebv'] = ebv

#c.transform_t
#text_df = pd.DataFrame(data=data, index=data['field_id'], columns=['ra','dec','l','b', 'eclLon', 'eclLat','grid_id'])
#text_df = pd.DataFrame(data=data, index=data['field_id'], columns=['ra','dec', 'ebv', 'l','b', 'eclLon', 'eclLat','entry'])

/Users/frostig/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]


In [12]:
# SAVE FIELDS

#text_df.to_csv(path_or_buf='../data/WINTER_fields.csv')
#text_df = pd.DataFrame(data=data, index=data['field_id'],
#                       columns=['field_id','ra','dec','l','b', 'eclLon', 'eclLat', 'grid_id'])
#np.savetxt(r'../data/WINTER_fields.txt', text_df, 
#           header=' ID         RA        Dec       Gal Long    Gal Lat    Ecl Long   Ecl Lat     Grid', 
#           fmt='%10.5f')

text_df = pd.DataFrame(data=data, index=data['field_id'],
                       columns=['field_id','ra','dec', 'ebv', 'l','b', 'eclLon', 'eclLat', 'entry'])
np.savetxt('SUMMER_fields.txt', text_df, 
           header='ID         RA         Dec       Ebv      Gal Long  Gal Lat    Ecl Long  Ecl Lat   Entry', 
           fmt=['%06d', '%10.5f', '%10.5f', '%1.3f', '%10.5f', '%10.5f','%10.5f', '%10.5f','% 4d'])